作者：Piper蛋窝  
链接：https://www.zhihu.com/question/662575933/answer/57725556297  

相信不少 Python 开发者都听说过 __slots__，知道它可以帮助节省内存。但你是否思考过它背后的原理，以及在实际开发中的其他妙用？让我们一起深入探讨。从一个性能问题说起。假设你的一个系统需要处理大量的订单对象：

```
class Order:
    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity
```

### 创建100万个订单对象
orders = [Order(i, "BTC", 30000, 1) for i in range(1_000_000)]  
看起来很普通的代码，但当你用内存分析工具一看，这些对象占用的内存可能远超预期。为什么？__dict__ 的开销在 Python 中，普通类的实例属性都存储在 __dict__ 字典中。这种设计非常灵活，允许我们动态添加属性：order = Order(1, "BTC", 30000, 1)  
order.new_field = "动态添加的字段"  # 完全合法  
但这种灵活性是有代价的：每个实例都要维护一个字典。字典本身为了支持快速查找，会预分配一定的空间字典的开销在对象数量大时会累积成可观的内存消耗。   
__slots__ 登场让我们改造一下 Order 类：
```
class Order:
    __slots__ = ['order_id', 'symbol', 'price', 'quantity']
    
    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity
```  
这个改动带来了什么变化？内存占用显著降低（通常可以节省 30% 到 50% 的内存）属性访问速度提升（因为不需要字典查找）代码更加"显式"，所有可能的属性一目了然。 
### __slots__ 的工作原理
当我们使用 __slots__ 时，Python 会：在类级别创建一个固定的内存布局，类似 C 语言中的结构体不再为实例创建 __dict__ 和 __weakref__ 属性（除非显式添加到 __slots__ 中）将属性直接存储在预分配的固定大小的数组中，而不是字典里这带来了两个直接的好处：属性访问更快：直接通过数组偏移量访问，不需要哈希查找内存占用更少： 没有 __dict__ 的开销（每个实例至少节省一个字典的内存）属性存储更紧凑（类似 C 结构体）没有哈希表的空间预留让我们用代码验证这些优势：

In [4]:
# https://www.zhihu.com/question/662575933/answer/57725556297

import sys
import time
import tracemalloc


class OrderWithDict:
    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity


class OrderWithSlots:
    __slots__ = ['order_id', 'symbol', 'price', 'quantity']
    
    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity


def measure_memory_and_speed(cls, n_objects=1_000_000):
    # 启动内存跟踪
    tracemalloc.start()
    
    # 创建对象
    start_time = time.time()
    objects = [cls(i, "BTC", 30000, 1) for i in range(n_objects)]
    creation_time = time.time() - start_time
    
    # 测量内存
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # 测试属性访问速度
    start_time = time.time()
    for obj in objects:
        _ = obj.order_id
        _ = obj.symbol
        _ = obj.price
        _ = obj.quantity
    access_time = time.time() - start_time
    
    return {
        "内存占用(MB)": peak / 1024 / 1024,
        "对象创建时间(秒)": creation_time,
        "属性访问时间(秒)": access_time
    }


def main():
    # 测试普通类
    print("测试普通类:")
    dict_results = measure_memory_and_speed(OrderWithDict)
    for k, v in dict_results.items():
        print(f"{k}: {v:.2f}")
    
    print("\n测试使用 __slots__ 的类:")
    slots_results = measure_memory_and_speed(OrderWithSlots)
    for k, v in slots_results.items():
        print(f"{k}: {v:.2f}")
    
    # 计算差异百分比
    print("\n性能提升:")
    for k in dict_results:
        improvement = (dict_results[k] - slots_results[k]) / dict_results[k] * 100
        print(f"{k}: 提升 {improvement:.1f}%")

    # 展示单个对象的大小差异
    normal_obj1 = OrderWithDict(1, "BTC", 30000, 1)
    normal_obj2 = OrderWithDict(1, "BTCasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfadsfasdfasdfasdfasdfasdf", 300003000030000300003000030000300003000030000300003000030000300003000030000300003000030000300003000030000, 1)
    slots_obj = OrderWithSlots(1, "BTC", 30000, 1)
    print(f"\n单个对象大小对比:")
    print(f"普通对象1: {sys.getsizeof(normal_obj1)} bytes")
    print(f"普通对象1的__dict__: {sys.getsizeof(normal_obj1.__dict__)} bytes")
    print(f"普通对象1总大小: {sys.getsizeof(normal_obj1) + sys.getsizeof(normal_obj1.__dict__)} bytes")
    print(f"普通对象2: {sys.getsizeof(normal_obj2)} bytes")
    print(f"普通对象2的__dict__: {sys.getsizeof(normal_obj2.__dict__)} bytes")
    print(f"普通对象2总大小: {sys.getsizeof(normal_obj2) + sys.getsizeof(normal_obj2.__dict__)} bytes")
    print(f"Slots对象: {sys.getsizeof(slots_obj)} bytes")
    try:
        print(f"Slots对象的__dict__: {sys.getsizeof(slots_obj.__dict__)} bytes")
    except AttributeError as e:
        print(f"Slots对象没有__dict__属性：{e}")

if __name__ == "__main__":
    main()

测试普通类:
内存占用(MB): 179.72
对象创建时间(秒): 2.49
属性访问时间(秒): 0.08

测试使用 __slots__ 的类:
内存占用(MB): 95.79
对象创建时间(秒): 1.62
属性访问时间(秒): 0.07

性能提升:
内存占用(MB): 提升 46.7%
对象创建时间(秒): 提升 34.9%
属性访问时间(秒): 提升 11.5%

单个对象大小对比:
普通对象1: 48 bytes
普通对象1的__dict__: 104 bytes
普通对象1总大小: 152 bytes
普通对象2: 48 bytes
普通对象2的__dict__: 104 bytes
普通对象2总大小: 152 bytes
Slots对象: 64 bytes
Slots对象没有__dict__属性：'OrderWithSlots' object has no attribute '__dict__'
